In [70]:
# Convert ts file to json

import re

txt = ""
with open("learnsets.ts") as file:
    txt = file.read()

txt = re.split("=", txt)[1]
txt = re.sub("//.*", "", txt)
txt = re.sub("(\w+):", "\"\g<1>\":", txt)
txt = re.sub(",(\s*)(]|})", "\g<1>\g<2>", txt)
txt = re.sub(";", "", txt)

with open("learnsets.json", "w") as file:
    file.write(txt)


In [71]:
# Convert json to df

import json
import pandas as pd

learnsets_json = {}
with open("learnsets.json") as file:
    learnsets = json.load(file)

learnsets_dict = {
    "pokemon": [],
    "move": [],
    "learn_method": []
}
for pokemon, pokemon_data in learnsets.items():
    if "learnset" not in pokemon_data:
        # print(f"Skipped {pokemon}")
        continue
    for move, learn_methods in pokemon_data["learnset"].items():
        count = len(learn_methods)
        learnsets_dict["pokemon"].extend([pokemon] * count)
        learnsets_dict["move"].extend([move] * count)
        learnsets_dict["learn_method"].extend(learn_methods)

df = pd.DataFrame(learnsets_dict)
df

,pokemon,move,learn_method
0,missingno,blizzard,3L1
1,missingno,bubblebeam,3L1
2,missingno,cut,3L1
3,missingno,doubleedge,3L1
4,missingno,earthquake,3L1
...,...,...,...
298737,saharaja,thief,8M
298738,saharaja,thief,8L1
298739,saharaja,watergun,9L1
298740,saharaja,watergun,8L1


In [87]:
# Parse learn methods data

import pandas as pd

key = {
    "D": "Dream World",
    "E": "Egg",
    "L": "Level Up",
    "M": "TM",
    "R": "Restricted",
    "S": "Event",
    "T": "Tutor",
    "V": "Transfer from Virtual Console or Let's Go"
}

df[["gen", "lv"]] = df.learn_method.str.split("\D", expand=True, regex=True)
df["method"] = df.learn_method.str.replace("\d", "", regex=True).map(key)
df.loc[df.method == "Event", "lv"] = ""
df.drop("learn_method", axis=1, inplace=True)
df.to_csv("learnsets.csv", index=False)
df


,pokemon,move,gen,lv,method
0,missingno,blizzard,3,1,Level Up
1,missingno,bubblebeam,3,1,Level Up
2,missingno,cut,3,1,Level Up
3,missingno,doubleedge,3,1,Level Up
4,missingno,earthquake,3,1,Level Up
...,...,...,...,...,...
298737,saharaja,thief,8,,TM
298738,saharaja,thief,8,1,Level Up
298739,saharaja,watergun,9,1,Level Up
298740,saharaja,watergun,8,1,Level Up
